<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel: Python 3 (ipykernel)
</div>

## Lab 1: Setup a LLM Playground on SageMaker Studio

---

__Large Language Model (LLM) with `Llama2`, `LangChain`, and `Streamlit`.__

In this lab, we learn how to use SageMaker to download, provision, and send prompts to a Large Language Model, `Llama 2`. We create an agent using `LangChain`, and tie everything together by creating a UI and text input using `Streamlit` to make our own hosted chatbot interface.

## Contents

- [Model License information](#Model-License-information)
- [Download and host Llama2 model](#Download-and-host-Llama2-model)
  - [Set up](#Set-up)
- [Sending prompts](#Sending-prompts)
  - [Supported Parameters](#Supported-Parameters)
  - [Notes](#Notes)
- [Building an agent with LangChain](#Building-an-agent-with-LangChain)
- [LangChain Tools](#LangChain-Tools)
- [Developing and deploying the UI with Streamlit](#Developing-and-deploying-the-UI-with-Streamlit)
- [Tearing down resources](#Tearing-down-resources)

### Model License information
---

To perform inference on these models, you need to pass `custom_attributes='accept_eula=true'` as part of header. This means you have read and accept the end-user-license-agreement (EULA) of the model. EULA can be found in model card description or from https://ai.meta.com/resources/models-and-libraries/llama-downloads/. By default, this notebook sets `custom_attribute='accept_eula=false'`, so all inference requests will fail until you explicitly change this custom attribute.

Note: Custom_attributes used to pass EULA are key/value pairs. The key and value are separated by `'='` and pairs are separated by `';'`. If the user passes the same key more than once, the last value is kept and passed to the script handler (i.e., in this case, used for conditional logic). For example, if `'accept_eula=false; accept_eula=true'` is passed to the server, then `'accept_eula=true'` is kept and passed to the script handler.

---

In [ ]:
from ipywidgets import Dropdown

eula_dropdown = Dropdown(
    options=["True", "False"],
    value="False",
    description="**Please accept Llama2 EULA to continue:**",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(eula_dropdown)

In [ ]:
custom_attribute = f'accept_eula={eula_dropdown.value.lower()}'
print(f"Your Llama2 EULA attribute is set to:", custom_attribute)

### Connect to a Hosted Llama2 Model
---

#### Set up

We begin by installing and upgrading necessary packages. Restart the kernel after executing the cell below for the first time.

In [ ]:
!pip install --upgrade langchain  typing_extensions==4.7.1 streamlit wikipedia -q

In [ ]:
from IPython.display import display, Markdown

#### Connect to an Hosted Llama2 Model

In [ ]:
endpoint_name = "meta-llama2-13b-neuron-chat-tg-ep" 
boto_region = "us-west-2"

In [ ]:
import boto3
import sagemaker
from sagemaker import serializers, deserializers

sess = sagemaker.session.Session(boto_session=boto3.Session(region_name=boto_region))
smr_client = boto3.client("sagemaker-runtime", region_name=boto_region)

pretrained_predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

### Sending prompts
---

Next, we invoke the endpoint hosting our Llama 2 LLM with some queries. To guess the best results, however, it is important to be aware of the adjustable parameters of this model.

#### Supported Parameters
This model supports the following inference payload parameters:

* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches max_new_tokens. If specified, it must be a positive integer.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.

We'll begin with 512, 0.9, and 0.6 for these respectively, though feel free to alter these are we do to see how this may affect the LLM output.

You may specify any subset of the parameters mentioned above while invoking an endpoint. 

#### Notes
- If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.
- In order to support a 4k context length, this model has restricted query payloads to only utilize a batch size of 1. Payloads with larger batch sizes will receive an endpoint error prior to inference.
- This model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and alternating (u/a/u/a/u...).

In [ ]:
def set_llama2_params(
    max_new_tokens=1000,
    top_p=0.9,
    temperature=0.6,
):
    """ set Llama2 parameters """
    llama2_params = {}
    
    llama2_params['max_new_tokens'] = max_new_tokens
    llama2_params['top_p'] = top_p
    llama2_params['temperature'] = temperature
    return llama2_params

In [ ]:
def print_dialog(inputs, payload, response):
    dialog_output = []
    for msg in inputs:
        dialog_output.append(f"**{msg['role'].upper()}**: {msg['content']}\n")
    dialog_output.append(f"**ASSISTANT**: {response['generated_text']}")
    dialog_output.append("\n---\n")
    
    display(Markdown('\n'.join(dialog_output)))

In [ ]:
from typing import Dict, List

def format_messages(messages: List[Dict[str, str]]) -> List[str]:
    """
    Format messages for Llama-2 chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]
    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)
    
def send_prompt(params, prompt, instruction=""):
    
    custom_attributes=custom_attribute

    # default 'system', 'user' and 'assistant' prompt format
    base_input = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": prompt},
    ]
    # convert s/u/a format all-string <<SYS>> [INST] prompt format
    optz_input = format_messages(base_input)

    payload = {
        "inputs": optz_input,
        "parameters": params
    }
    response = pretrained_predictor.predict(payload, custom_attributes=custom_attribute)
    print_dialog(base_input, payload, response)
    return payload, response

With functions defined for the printing of the dialog and the prompt sending, let's begin sending queries to our Llama 2 LLM!

Note that we can also adjust the parameters supplied of the model, which we do by altering the `top_p parameter`.

In [ ]:
%%time
params = set_llama2_params(top_p=0.4)
payload, response = send_prompt(params, prompt="What is the recipe of a pumpkin pie?")

In [ ]:
%%time
params = set_llama2_params(top_p=0.6)
payload, response = send_prompt(params, prompt="How do I learn to play the guitar?", instruction="always answer with Haiku")

In [ ]:
%%time
params = set_llama2_params(top_p=0.8)
payload, response = send_prompt(params, prompt="What's a good strategy for chess?", instruction="always answer with emojis")

In [ ]:
%%time
params = set_llama2_params(top_p=0.6)
tokyo_payload, tokyo_response = send_prompt(params, prompt="What are the top 5 things to do in Tokyo?")

Because we are interacting with the llama2 **chat** LLM, we can input a previous prompt with a further question in a conversation manner. 

Also, because we are capturing the payload and response for each inference to our endpoint, we can feed this back into our LLM as part of our next prompt, in order to continue the conversation. In the following output we can see the requests and repsonses from the user, and the assistant:

In [ ]:
%%time
base_input = [
    {
        "role": "user", 
        "content": "What are the top 5 things to do in Tokyo?"},
    {
        "role": "assistant",
        "content": tokyo_response['generated_text'],
    },
    {
        "role": "user", 
        "content": "What is so great about #1?"  # <<---- Your follow up question here!
    },
]
optz_input = format_messages(base_input)

payload = {
    "inputs": optz_input,
    "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
}
response = pretrained_predictor.predict(payload, custom_attributes=custom_attribute)
print_dialog(base_input, payload, response)

---
### Building an agent with LangChain
---

We now have a LLM that can continue conversations in a chat interface! However, there is a more effective option than manually capturing the request and response for each inference request and feeding this back into the model.

[LangChain](https://www.langchain.com/) is a framework that helps us simplify this process. We can use LangChain to send prompts to our LLM, store chat histroy, and feed this back into the model in order to have a conversation.

LangChain also allows us to define a content header to transform the inputs and outputs to the LLM, which we will do in the next cell.

In [ ]:
from typing import Dict
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt, model_kwargs):
        base_input = [{"role" : "user", "content" : prompt}]
        optz_input = format_messages(base_input)
        input_str = json.dumps({
            "inputs" : optz_input, 
            "parameters" : {**model_kwargs}
        })
        return input_str.encode('utf-8')
    
    def transform_output(self, output):
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_text"]

We can then pass the SageMaker endpoint we previoiusly provisioned into a LangChain `SageMaker Endpoint` object, which allows LangChain to interact with out Llama 2 LLM. We are also passing in parameters which we defined previously.

In [ ]:
import json
from sagemaker import session

content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=pretrained_predictor.endpoint_name, 
     region_name=session.Session().boto_region_name, 
     model_kwargs={"max_new_tokens": 700, "top_p": 0.9, "temperature": 0.6},
     endpoint_kwargs={"CustomAttributes": custom_attribute},
     content_handler=content_handler
 )

We can now create a chat prompt template that LangChain will pass to our LLM. The LangChain [ChatPromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/#chatprompttemplate) object allows us to do this.

We also have [ConversationBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html) and [LLMChain](https://docs.langchain.com/docs/components/chains/llm-chain) objects. The former allows to store the conversation memory, and the latter brings together the Chat Prompt Template, LLM, and Conversation Buffer Memory. We also set `verbose` to `True`, allowing us, in this case, to see the conversation history up until this point.

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# Prompt 
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Assistant is a chatbot having a conversation with a human. Assistant is informative and polite, and only answers the question asked."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we set`return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

Now we have our conversation LLM Chain, LangChain will pass our query, as well as the history and chat prompt template, to the LLM. This is great for a chatbot interface, as we'll demonstrate now. For each of the following three cells' output, you'll notice the conversation history after the text `Entering new LLMChain chain...`, and the query response after `Finished chain.`

In [ ]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
def simple_conversation(question):
    print(conversation({"question": question})['text'])

In [ ]:
simple_conversation('hi!')

In [ ]:
simple_conversation("How can I travel from New York to Los Angeles?")

In [ ]:
simple_conversation("Can you tell me more about the first option?")

___
### LangChain Tools
___

LangChain, further, has [tools](https://python.langchain.com/docs/modules/agents/tools/) which it can use to send API requests to perform various tasks which it may not have been able to do in isolation, such as make a search request or check the weather. Today, we will be using a math, and a Wikipedia tool, though please see a more complete list [here](https://js.langchain.com/docs/api/tools/). It is also possible to [create your own tool](https://python.langchain.com/docs/modules/agents/tools/custom_tools).

We also use LangChain [agents](https://docs.langchain.com/docs/components/agents/). Agents are especially powerful where there is not a predetermined chain of calls, like we've had above so far. It is possible to have an unknown chain that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call. An agent could call multiple LLM Chains that we defined above, each with their own tools. They can also be extended with custom logic to allow for retries, and error handling.

Defining our two tools, as well as the LangChain agent, will give us a model that will able to determine whether it needs to use Wikipedia or a math tool, or whether it is able to answer a question on its own. If it needs the tool, it will make a request to the tool, receive the response, and then return that response to the user.

We also define an Output Parser, which is a method of parsing the output from the prompt. If the LLM produces output uses certain headers, we can enable complex interactions where variables are generated by the LLM in their response and passed into the next step of the chain.

In [ ]:
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.agents import AgentOutputParser, load_tools
from langchain.schema import AgentAction, AgentFinish

tools = load_tools(["llm-math", "wikipedia"], llm=llm)

class OutputParser(AgentOutputParser):

    def parse(self, response):
        try:
            parsed_response=parse_json_markdown(response)
            step, step_input = parsed_response["step"], parsed_response["step_input"]
            if step == "Final Answer":
                return AgentFinish({"output": step_input}, response)
            else:
                return AgentAction(step, step_input, response)
        except:
            return AgentFinish({"output": response}, response)
        
    def get_format_instructions(self):
        return FORMAT_INSTRUCTIONS

parser = OutputParser()

We initialize the agent with the tools we have defined above, the [agent type](https://python.langchain.com/docs/modules/agents/agent_types/), as well as the LLM, memory, and output parser we defined above. Again we set `Verbose` to `True`, which in this case will allow us to see if and how the agent calls a tool it has access to.

In [ ]:
from langchain.agents import initialize_agent

# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    memory=memory,
    llm=llm,
    tools=tools,
    verbose=True,
    agent_kwargs={
        "output_parser": parser
    }
)

We also provide a background prompt to the model. This provides the LLM with instructions of the tools it has access to, when to use which, and how to use each. This allows the LLM to firstly know when to use a tool (as opposed to answering in isolation 'by itself'), but also allows the LangChain agent to create a request to the tool the LLM has identified, before returning to the LLM to respond in a natural language way.

In [ ]:
system_message = """

<>\n Assistant is designed to build JSON and answer a wide variety of User questions.

Assistant must use JSON strings that contain "step" and "step_input" parameters. All of Assistant's communication is performed using this JSON format.

Tools available to Assistant are:

- "Wikipedia": Useful when you need a summary of a person, place, historical event, or other subject. Input is typically a noun, like a person, place, historical event, or another subject.
  - To use the wikipedia tool, Assistant should format the JSON like the following before getting the response and returning to the user:
    ```json
    {{"step": "Wikipedia",
      "step_input": "Statue of Liberty"}}
    ```
- "Calculator": Useful for when you need to answer questions about math. Input is one or more number combined with one or more math operations (addition, subtraction, multiplation, division, square root, exponetnial, and more).
  - To use the calculator tool, Assistant should format the JSON like the following so before getting the response and returning to the user:
    ```json
    {{"step": "Calculator",
      "step_input": "24*189"}}
    ```

Here is the set of previous interactions between the User and Assistant:

User: Hi!
Assistant: ```
{{"step": "Final Answer",
 "step_input": "Hello! How can I assist today?"}}
```
User: What is 9 cubed?
Assistant: ```
{{"step": "Calculator",
 "step_input": "9**3"}}
```
User: 729
Assistant: ```
{{"step": "Final Answer",
 "step_input": "The answer to your question is 729."}}
```
User: Can you tell me about the Statue of Liberty?
Assistant: ```
{{"step": "Wikipedia",
 "step_input": "Statue of Liberty"}}
```
User: The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor in New York City, in the United States. The copper statue, a gift from the people of France, was designed by French sculptor Frédéric Auguste Bartholdi and its metal framework was built by Gustave Eiffel.
Assistant: ```
{{"step": "Final Answer",
 "step_input": "Sure! The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor in New York City, in the United States. The copper statue, a gift from the people of France, was designed by French sculptor Frédéric Auguste Bartholdi and its metal framework was built by Gustave Eiffel."}}
```
User: What is the square root of 81?
Assistant: ```
{{"step": "Calculator",
 "step_input": "sqrt(81)"}}
```
User: 9
Assistant: ```
{{"step": "Final Answer",
 "step_input": "The answer to your question is 9."}}
```

Assistant should use a tool only if needed, but if the assistant does use a tool, the result of the tool must always be returned back to the user with a "Final Answer" step. Only use the calculator if the 'step_input' includes numbers. \n<>\n\n
"""

few_shot = agent.agent.create_prompt(
    system_message=system_message,
    tools=tools
)
agent.agent.llm_chain.prompt = few_shot

agent.agent.llm_chain.prompt.messages[2].prompt.template = "[INST] Respond in JSON with 'step' and 'step_input' values until you return an 'step': 'Final Answer', along with the 'step_input'. [/INST] \nUser: {input}"

We can now send some prompts to the LLM and see when/how it uses the tools!

In [ ]:
def agent_conversation(question):
    print(agent(question))

In [ ]:
agent_conversation('how are you?')

In [ ]:
agent_conversation("Tell me about the Empire Statue Building")

In [ ]:
agent_conversation("What is the square root of 64?")

In [ ]:
response = agent_conversation("can you divide the answer to this last question by five?")

___
### Developing and deploying the UI with Streamlit
___

Let's bring all of this together and host our chatbot interface!

For this we will use [Streamlit](https://streamlit.io/). Streamlit is an open-source Python library that allows you to create and deploy web applications. It can be deployed from our local machine, or from the Cloud. Today, we will deploy it directly from SageMaker Studio.

The file `app.py` brings together all of what we have discussed so far. It initializes a LanChain Agent, with the tools and conversation memory we spoke about previously. It connects to our same Llama 2 LLM.

The majority of this code you will be familiar with from the notebook so far. The rest uses the [Streamlit library](https://docs.streamlit.io/library/api-reference), as well as [LangChain Streamlit packages](https://python.langchain.com/docs/integrations/memory/streamlit_chat_message_history). 

It is one of the last lines of the file, `response = agent(prompt, callbacks=[st_cb])` that sends the prompt to the agent, as well as specifies the [StreamlitCallbackHandler](https://python.LangChain.com/docs/integrations/callbacks/streamlit) which can display the reasoning and actions in the streamlit app. By default we are not showing this in the conversation, and have a regex that filers out too much of the conversation history and thought process, though in order to see comment out the line at the end `response = re.sub("\{.*?\}","",response["output"])`.

We are also using [st.chat_message](https://docs.streamlit.io/library/api-reference/chat/st.chat_message) to handle the chat message container, and [st.write](https://docs.streamlit.io/library/api-reference/write-magic/st.write) to return this, along with the previous conversation, back to the UI.

We can [build Streamlit apps in SageMaker Studio](https://aws.amazon.com/blogs/machine-learning/build-streamlit-apps-in-amazon-sagemaker-studio/). We will do this by hosting the app on the Jupyter Server. 

Firstly, let's write the output of our SageMaker endpoint to a text file so it can be read by the `app.py`:

In [ ]:
f = open("../studio-local-ui/endpoint_name.txt", "w")
f.write(pretrained_predictor.endpoint_name)
f.close()

In [ ]:
f = open("../studio-local-ui/custom_attribute.txt", "w")
f.write(custom_attribute)
f.close()

Run the following cells marked with `%%bash`, these cells will install a few packages in your conda environment and spin up a new Streamlit UI that's accessible from the URL described below.

In [ ]:
%%bash
cd ../studio-local-ui
python3 -m pip install -r requirements.txt -q

In [ ]:
%%bash
cd ../studio-local-ui
streamlit run chat_app.py --server.runOnSave true --server.port 8501 > /dev/null

<div style="background-color: #6bb07e; border-left: 5px solid #6bb07e; padding: 10px; color: black;">
    - Navigate to: https://example.studio.us-west-2.sagemaker.aws/jupyterlab/default/proxy/8502/
</div>

<div style="background-color: #6bb07e; border-left: 5px solid #6bb07e; padding: 10px; color: black;">
    <i>- Replace "example" with your your current url host `https://use_this_host.studio.us-west-2...`</i>
</div>